# Финальая попытка улучшить показатели
Максимум, что удалось выжать ранее
- RMSE: 23465.1123
- R² (коэффициент детерминации): 0.9211
- Средний RMSE на кросс-валидации: 25687.30318717033

В этой версии переймем лучшие практики из прошлых подходов: 
- удаление ненужных признаков интуитивных;
- внедрим IterativeImputer, который использует RandomForestRegression для обучения модели для заполнения пропущенных числовых фич;
- *из нового* - попробуем поэкспериментировать еще с фича инжинирнгом, но уже не просто что-то удалить или создать или запустить SequentialFeatureSelector на 20 часов для подбора наилучшей комбинации фич, который не дал ничего - а логически проанализировать фичи, возможно, что-то добавить, объединить, как-то более хитро нормализовать и т.д.
- поэкспериментируем с борьбой с переобучением (ограничить максимальную глубину деревьев (например, 6-8 вместо 10-12 по умолчанию); параметр L2-регуляризации листьев (l2_leaf_reg) добавляет штраф на большие значения предсказаний в листьях; rsm (Random Subspace Method) = 0.8)
- детектор переобучения – можно указать параметр od_type (например, Iter или IncToDec) и od_wait, чтобы модель автоматически прекращала обучение, как только ошибка на валидации начинает расти (ранняя остановка)
- сгенерировать дополнительные синтетические данные: 1) Добавление шума к числовым признакам (Jittering) - к каждому числовому признаку можно добавить небольшое случайное гауссово отклонение (с очень малой дисперсией, чтобы не исказить порядок величин). 2) SMOTE (Synthetic Minority Over-sampling Technique) - генерация новых объектов как интерполяция между ближайшими соседями - пытаться генерировать новый дом как среднее (или случайную комбинацию) двух похожих домов из обучающей выборки, а цену назначить промежуточную. 3) Генеративные модели данных - Conditional Tabular GAN (CTGAN), генерative adversarial network для таблиц. CTGAN умеет создавать синтетические табличные данные, сохраняя распределения как категориальных, так и числовых признаков (можно обучить CTGAN на имеющихся 2000 объектов и сгенерировать, скажем, ещё 1000 «новых» домов. Далее объединить их с реальными для обучения CatBoost.)

Важно(!) Правильная валидация – краеугольный камень при работе с небольшим датасетом. 
С маленькой выборкой целесообразно запускать многократную (repeated) кросс-валидацию – например, 5-кратную CV повторить 2-3 раза с разными разбиениями и усреднить результат. 

In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import mlflow
import mlflow.sklearn
import pandas as pd

from catboost import CatBoostRegressor
from category_encoders import TargetEncoder
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PowerTransformer, RobustScaler, QuantileTransformer, PolynomialFeatures, KBinsDiscretizer, FunctionTransformer

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from utils.data_manager import DataManager
from utils.model_manager import ModelManager
from utils.synth_generator import SimpleSyntheticGenerator
from utils.syth_generator_gaussian import CombinedSyntheticGenerator
from utils.synth_generator_CTGAN import OptimizedCTGAN

In [2]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [3]:
dm = DataManager()
mm = ModelManager()

mlflow.sklearn.autolog()

## 1. Загрузка данных

In [4]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

## 2. Предобработка данных 

### 2.1. Удаление ненужных столбцов

In [5]:
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)
bad_columns

['Id',
 'LotShape',
 'LandContour',
 'LotConfig',
 'LandSlope',
 'MiscFeature',
 'MiscVal']

In [6]:
train_data = train_data.drop(columns=bad_columns)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition1,Condition2,...,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,AllPub,CollgCr,Norm,Norm,...,0,0,0,NaN,NaN,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,AllPub,Veenker,Feedr,Norm,...,0,0,0,NaN,NaN,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,AllPub,CollgCr,Norm,Norm,...,0,0,0,NaN,NaN,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,AllPub,Crawfor,Norm,Norm,...,0,0,0,NaN,NaN,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,AllPub,NoRidge,Norm,Norm,...,0,0,0,NaN,NaN,12,2008,WD,Normal,250000


### Пробуем нагенерировать синтетические данные

In [7]:
# # Инициализация и обучение генератора
# generator = SimpleSyntheticGenerator()
# generator.fit(train_data)
# 
# # Генерация синтетических данных
# synthetic_df = generator.generate(count=1500)
# 
# # Сохранение результатов
# synthetic_df.to_csv('synthetic_data.csv', index=False)
# 
# sdf = pd.read_csv('synthetic_data.csv')
# # Объединяем с реальными данными
# train_data = pd.concat([train_data, sdf], ignore_index=True)

# Очень плохие результаты по коду выше

In [8]:
# # Инициализация и обучение генератора
# generator = CombinedSyntheticGenerator(use_gaussian_copula=True)
# generator.fit(train_data)
# 
# # Генерация синтетических данных
# synthetic_df = generator.generate(count=500)
# 
# # Сохранение результатов
# synthetic_df.to_csv('synthetic_data2.csv', index=False)
# 
# sdf = pd.read_csv('synthetic_data2.csv')
# # Объединяем с реальными данными
# train_data = pd.concat([train_data, sdf], ignore_index=True)
# train_data.shape

# Решение выше дало потрясающие результаты RMSE, R2 без кроссвалидации
# RMSE: 16184.623736498073
# R²: 0.9581875788511584
# и даже с кросс-валидацией чудно
# Средний RMSE на многократной кросс-валидации: 14632.7328
# Стандартное отклонение RMSE: 1422.7506
# Но сабмишн на кагл показал ужасные результаты, получается - жесткий оверфиттинг. Что очевидно, учитывая, что 3к данных нагенерили.

In [9]:
# # Инициализация генератора с настроенными параметрами
# ctgan_generator = OptimizedCTGAN(
#     categorical_threshold=15,  # Настройте под ваши данные
#     epochs=50,                 # Начните с малого значения
#     batch_size=64,             # Оптимальный размер для стабильности
#     generator_dim=(64, 64),    # Уменьшенные размеры для скорости
#     discriminator_dim=(64, 64),
#     embedding_dim=32,
#     cuda=False,                # True если есть GPU
#     verbose=True
# )
# 
# # Обучение модели
# ctgan_generator.fit(train_data)
# 
# # # Генерация данных
# # synthetic_df = ctgan_generator.generate(count=1500)
# # 
# # # Сохранение и использование
# # synthetic_df.to_csv('synthetic_data_ctgan.csv', index=False)
# # combined_data = pd.concat([train_data, synthetic_df], ignore_index=True)
# # print(f"Размер объединенных данных: {combined_data.shape}")

# Это решение так и не вышло запустить по причине того, что все зависает на 0 итерации.

In [10]:
RANDOM_STATE = 42

In [11]:
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)

(1460, 73) (1460,)


In [12]:
# # Пытаемся еще пофичаинженерить
# X['TotalSF'] = X['1stFlrSF'] + X['2ndFlrSF'] + X['TotalBsmtSF']
# X['Age'] = X['YrSold'] - X['YearBuilt']
# X['QualityIndex'] = X['OverallQual'] * X['OverallCond']
# X['HasPool'] = (X['PoolArea'] > 0).astype(int)
# X['Remodeled'] = (X['YearRemodAdd'] != X['YearBuilt']).astype(int)
# X['TotalBathrooms'] = X['FullBath'] + 0.5 * X['HalfBath'] + X['BsmtFullBath'] + 0.5 * X['BsmtHalfBath']
# X.shape
# # Не дало улучшений

In [13]:
# Еще пытаемся пофичаинженерить
# X['Neighborhood_Qual'] = X['Neighborhood'] + '_' + X['OverallQual'].astype(str)
# X['BldgType_Condition'] = X['BldgType'] + '_' + X['OverallCond'].astype(str)

# И еще
# Полезная жилая площадь на всех уровнях
# X['UsefulSF'] = X['GrLivArea'] + X['TotalBsmtSF'] - X['LowQualFinSF']
# 
# # Доля жилой площади в общей площади
# X['LivingAreaRatio'] = X['GrLivArea'] / X['LotArea']
# 
# # Отношение гаража к дому
# X['GarageRatio'] = X['GarageArea'] / X['GrLivArea']
# 
# # Отношение площади террас и крылец к жилой площади
# X['PorchDeckArea'] = (X['WoodDeckSF'] + X['OpenPorchSF'] + X['EnclosedPorch'] +
#                       X['3SsnPorch'] + X['ScreenPorch'])
# X['PorchDeckRatio'] = X['PorchDeckArea'] / X['GrLivArea']

# и Еще
# X['IsNewHouse'] = (X['YrSold'] - X['YearBuilt'] <= 1).astype(int)
# X['IsRecentRemodel'] = (X['YrSold'] - X['YearRemodAdd'] <= 5).astype(int)
# X['GarageAge'] = X['YrSold'] - X['GarageYrBlt']
# X['GarageAge'] = X['GarageAge'].fillna(X['GarageAge'].median())  # важно обработать NA

# и еще
X['Neighborhood_Zoning'] = X['Neighborhood'] + '_' + X['MSZoning']
X['SaleType_Condition'] = X['SaleType'] + '_' + X['SaleCondition']
# Получили прирост качества!

# и еще
# Объединяем несколько качественных признаков в единый числовой индекс:
quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}
X['TotalQualScore'] = (
    X['ExterQual'].map(quality_dict) +
    X['KitchenQual'].map(quality_dict) +
    X['BsmtQual'].map(quality_dict) +
    X['HeatingQC'].map(quality_dict) +
    X['GarageQual'].map(quality_dict) +
    X['FireplaceQu'].map(quality_dict)
)
# Получили значимый (-400 RMSE) прирост качества!

# и еще
# # Отношение площади террас и крылец к жилой площади
X['PorchDeckArea'] = (X['WoodDeckSF'] + X['OpenPorchSF'] + X['EnclosedPorch'] +
                      X['3SsnPorch'] + X['ScreenPorch'])
X['HasFireplace'] = (X['Fireplaces'] > 0).astype(int)
X['HasGarage'] = (~X['GarageType'].isna()).astype(int)
X['HasFence'] = (~X['Fence'].isna()).astype(int)
X['HasPorchDeck'] = (X['PorchDeckArea'] > 0).astype(int)
# Получили значимый (-400 RMSE) прирост качества!


In [14]:
# --- Функция для добавления полиномиальных признаков ---
# Она будет использоваться в FunctionTransformer
# def add_polynomial_features(df, poly_cols):
#     """
#     Добавляет полиномиальные признаки 3-й степени для указанных колонок.
#     Заполняет пропуски медианой перед генерацией.
#     Удаляет исходные колонки, использованные для генерации.
#     """
#     df_copy = df.copy()
#     # Убедимся, что все нужные колонки есть в DataFrame
#     cols_to_process = [col for col in poly_cols if col in df_copy.columns]
#     if not cols_to_process:
#          print("Warning: No columns found for polynomial feature generation.")
#          return df_copy # Возвращаем копию без изменений
# 
#     df_subset = df_copy[cols_to_process]
# 
#     # Заполняем пропуски медианой ТОЛЬКО для этих колонок
#     # Важно: медиану лучше считать на трейне и передавать сюда,
#     # но для простоты примера считаем на лету. Для пайплайна это ок.
#     for col in cols_to_process:
#          if df_subset[col].isnull().any():
#                median_val = df_subset[col].median()
#                df_subset[col] = df_subset[col].fillna(median_val)
# 
#     # Создаем объект PolynomialFeatures
#     poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
# 
#     # Обучаем и трансформируем подмножество данных
#     poly_features_generated = poly.fit_transform(df_subset)
# 
#     # Получаем имена новых признаков
#     poly_feature_names = poly.get_feature_names_out(cols_to_process)
# 
#     # Создаем DataFrame с новыми признаками
#     df_poly_features = pd.DataFrame(poly_features_generated, columns=poly_feature_names, index=df_copy.index)
# 
#     # Удаляем исходные колонки из копии
#     df_copy = df_copy.drop(columns=cols_to_process)
# 
#     # Объединяем исходный DataFrame (без старых колонок) с новыми полиномиальными
#     df_final = pd.concat([df_copy, df_poly_features], axis=1)
# 
#     print(f"Polynomial features added. Total columns now: {df_final.shape[1]}")
#     return df_final
# # Вроде дает небольшое улучшение RMSE

In [15]:
# # --- Winsorizing (Ограничение выбросов) ---
# Пример ДО встраивания в пайплайн
# Как выбрать колонки и встроить в пайплайн?
# Визуальный анализ: Построить гистограммы или box plot-ы для всех числовых признаков. Выбрать для Winsorizing только те, у которых явно видны длинные хвосты или отдельные точки далеко от основного распределения. GrLivArea, LotArea, TotalBsmtSF, 1stFlrSF, BsmtFinSF1 — частые кандидаты.
# 
# Подбор квантилей: Вместо фиксированных 1%/99%, попробовать другие значения (например, 0.5%/99.5% или 2%/98%) и посмотреть на кросс-валидации, какой диапазон лучше работает для конкретного признака или группы признаков.
# 
# Кастомный трансформер для пайплайна: Чтобы встроить Winsorizing в пайплайн sklearn и правильно обрабатывать данные при кросс-валидации (считать квантили на трейне, применять на тесте), нужно создать свой класс-трансформер.

# cols_to_winsorize = X.select_dtypes(include=['float64', 'int64']).columns # Пример колонки
# 
# for col_to_winsorize in cols_to_winsorize:
#     # Определяем квантили (например, 1% и 99%)
#     lower_quantile = X[col_to_winsorize].quantile(0.01)
#     upper_quantile = X[col_to_winsorize].quantile(0.99)
#     
#     print(f"\nИсходные мин/макс для '{col_to_winsorize}': {X[col_to_winsorize].min()}/{X[col_to_winsorize].max()}")
#     print(f"Нижний (1%) и верхний (99%) квантили: {lower_quantile:.2f} / {upper_quantile:.2f}")
#     
#     # Применяем ограничение с помощью .clip()
#     # Все значения ниже lower_quantile станут равны lower_quantile
#     # Все значения выше upper_quantile станут равны upper_quantile
#     X[col_to_winsorize + '_winsorized'] = X[col_to_winsorize].clip(lower=lower_quantile, upper=upper_quantile)
#     
#     print(f"Мин/макс для '{col_to_winsorize}_winsorized': {X[col_to_winsorize + '_winsorized'].min()}/{X[col_to_winsorize + '_winsorized'].max()}")
#     print("---")

In [16]:
# ОЧЕНЬ ВАЖНО убедиться, что набор признаков (X_submission) абсолютно совпадает с X по количеству и порядку колонок!
X = X.sort_index(axis=1)
feature_cols = X.columns

### 2.2. Нормализация данных через ColumnTransformer и Pipeline
В данном кейсе мы реализуем Заполнение числовых пропусков с помощью модели (Predictive imputation). Т.е. то, что пропущено в числовых признаках - будем заполнять не медианой или средним, а будем обучать модель, которая будет предсказывать пропуски (IterativeImputer + RandomForestRegressor)

In [17]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns

# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

In [18]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# --- Кастомный трансформер Winsorizer ---
class Winsorizer(BaseEstimator, TransformerMixin):
    """
    Применяет Winsorizing к указанным колонкам DataFrame.

    Parameters
    ----------
    quantile_range : tuple, optional (default=(0.01, 0.99))
        Кортеж с нижним и верхним квантилями для ограничения.
    columns : list
        Список названий колонок, к которым нужно применить Winsorizing.
    """
    def __init__(self, quantile_range=(0.01, 0.99), columns=None):
        self.quantile_range = quantile_range
        self.columns = columns
        self.limits_ = {} # Словарь для хранения вычисленных квантилей

    def fit(self, X, y=None):
        """
        Вычисляет квантили на обучающих данных.

        Parameters
        ----------
        X : pd.DataFrame
            Обучающие данные.
        y : None
            Игнорируется.

        Returns
        -------
        self
        """
        if self.columns is None:
            # Если колонки не указаны, применяем ко всем числовым
            self.columns = X.select_dtypes(include=np.number).columns.tolist()

        # Проверяем, что X - это DataFrame
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input X must be a pandas DataFrame")

        for col in self.columns:
            if col in X.columns:
                lower_quantile_val = X[col].quantile(self.quantile_range[0])
                upper_quantile_val = X[col].quantile(self.quantile_range[1])
                self.limits_[col] = (lower_quantile_val, upper_quantile_val)
            else:
                 print(f"Warning: Column '{col}' not found in DataFrame during fit.")
        return self

    def transform(self, X):
        """
        Применяет ограничение (clipping) к данным.

        Parameters
        ----------
        X : pd.DataFrame
            Данные для трансформации.

        Returns
        -------
        pd.DataFrame
            Трансформированные данные.
        """
        X_copy = X.copy()
        # Проверяем, что X - это DataFrame
        if not isinstance(X_copy, pd.DataFrame):
             raise ValueError("Input X must be a pandas DataFrame")

        for col in self.columns:
            if col in self.limits_: # Применяем только если квантили были посчитаны
                 if col in X_copy.columns:
                     lower_limit, upper_limit = self.limits_[col]
                     X_copy[col] = X_copy[col].clip(lower=lower_limit, upper=upper_limit)
                 else:
                     print(f"Warning: Column '{col}' not found in DataFrame during transform.")
            # Если колонки не было в fit, ничего не делаем с ней
        return X_copy
    

# --- Создаем трансформер на основе функции полиномиальной ---
# validate=False, т.к. функция меняет количество и названия колонок
# poly_features_cols = ['GrLivArea', 'TotalBsmtSF', 'YearBuilt', 'LotArea']  # Колонки для полиномиальных признаков
# polynomial_transformer = FunctionTransformer(
#     add_polynomial_features,
#     kw_args={'poly_cols': poly_features_cols}, # Передаем список колонок в функцию
#     validate=False
# )
# Кроме переобучение - ничего не дали полиномиальные признаки, хоть текущий RMSE улучшился сильно, а самбишн - плохой стал


# Пайплайн для числовых признаков (итеративное заполнение)
# Колонки, которые мы хотим "винсоризировать" (выбираем на основе анализа)
cols_to_winsorize = ['GrLivArea', 'LotArea', 'TotalBsmtSF', '1stFlrSF']
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
        max_iter=10,
        random_state=RANDOM_STATE
    )),
    # Применяем Winsorizer ПОСЛЕ импутации
    # ('winsorizer', Winsorizer(quantile_range=(0.01, 0.99), columns=cols_to_winsorize)),
    # Опционально: добавляем масштабирование ПОСЛЕ winsorizing
    # ('scaler', StandardScaler())
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    # ('target_enc', TargetEncoder(smoothing=10, min_samples_leaf=4, handle_unknown='value', handle_missing='value'))
])

# --- Объединяем препроцессоры ---
# Применяем к исходным числовым и категориальным колонкам
# Полиномиальные признаки будут добавлены ДО этого шага
preprocessor = ColumnTransformer(
    transformers=[
        # Применяем к исходным числовым колонкам
        ('num', numeric_transformer, numeric_columns),
        # Применяем к исходным категориальным колонкам
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
    verbose_feature_names_out=False # Чтобы имена колонок не менялись на 'num__colname' и т.д.
)

default_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
}
# --- Финальный пайплайн ---
# Добавляем polynomial_transformer как ПЕРВЫЙ шаг
final_pipeline = Pipeline([
    # ('poly_features', polynomial_transformer), # Шаг 1: Добавляем полиномиальные признаки
    ('preprocessing', preprocessor),          # Шаг 2: Применяем остальной препроцессинг
    ('model', CatBoostRegressor(**default_params))  # Шаг 3: Модель
])

### 2.3. Разбиение данных на обучающую и тестовую выборки

In [19]:
# # Один из самых эффективных приемов для задач прогнозирования цен на недвижимость: Выявление и устранение аномальных значений (Логарифмическая трансформация: Применение log1p() (натуральный логарифм от значения плюс 1) к ценам: Сжимает шкалу, приближая распределение к нормальному; Уменьшает влияние выбросов без их удаления) и пр.
# # Логарифмирование целевой переменной
# y_log = np.log1p(y)
# 
# # Разделение на обучающую и тестовую выборки
# X_train, X_test, y_log_train, y_log_test = train_test_split(
#     X, y_log, test_size=0.3, random_state=RANDOM_STATE
# )
# 
# # Обучение модели на логарифмированных ценах
# final_pipeline.fit(X_train, y_log_train)
# 
# # Предсказание (в логарифмической шкале)
# y_log_pred = final_pipeline.predict(X_test)
# 
# # Обратное преобразование для возврата к исходной шкале
# y_pred = np.expm1(y_log_pred)
# y_test_original = np.expm1(y_log_test)
# 
# # Расчет метрик на исходной шкале
# print("RMSE:", root_mean_squared_error(y_test_original, y_pred))
# print("R²:", r2_score(y_test_original, y_pred))

# Показатели не улучшились
# RMSE: 24355.361915626483
# R²: 0.9149934531746243


In [20]:
# # Попробуем другие преобразования Y
# y_sqrt = np.sqrt(y)
# 
# # Разделение на обучающую и тестовую выборки
# X_train, X_test, y_sqrt_train, y_sqrt_test = train_test_split(
#     X, y_sqrt, test_size=0.3, random_state=RANDOM_STATE
# )
# 
# # Обучение модели
# final_pipeline.fit(X_train, y_sqrt_train)
# 
# # Предсказание
# y_sqrt_pred = final_pipeline.predict(X_test)
# 
# # Обратное преобразование для возврата к исходной шкале
# y_pred = y_sqrt_pred ** 2
# y_test_original = y_sqrt_test ** 2
# 
# # Расчет метрик на исходной шкале
# print("RMSE:", root_mean_squared_error(y_test_original, y_pred))
# print("R²:", r2_score(y_test_original, y_pred))

# Показатели лучше, чем в log1p версии, но не улучшились от идеала
# RMSE: 24016.628174145026
# R²: 0.9173415479046967


In [21]:
# # Попробуем другие преобразования Y
# from scipy import stats
# from scipy.special import inv_boxcox
# y_boxcox, lambda_param = stats.boxcox(y)
# 
# # Разделение на обучающую и тестовую выборки
# X_train, X_test, y_bxcx_train, y_bxcx_test = train_test_split(
#     X, y_boxcox, test_size=0.3, random_state=RANDOM_STATE
# )
# 
# # Обучение модели
# final_pipeline.fit(X_train, y_bxcx_train)
# 
# # Предсказание 
# y_bxcx_pred = final_pipeline.predict(X_test)
# 
# # Обратное преобразование для возврата к исходной шкале
# y_pred = inv_boxcox(y_bxcx_pred, lambda_param)
# y_test_original = inv_boxcox(y_bxcx_test, lambda_param)
# 
# # Расчет метрик на исходной шкале
# print("RMSE:", root_mean_squared_error(y_test_original, y_pred))
# print("R²:", r2_score(y_test_original, y_pred))

# Еще хуже, чем у log1p показатели
# RMSE: 24708.20382416625
# R²: 0.9125125919582991

In [22]:
# Попробуем другие преобразования Y
# from sklearn.preprocessing import QuantileTransformer
# qt = QuantileTransformer(output_distribution='normal', random_state=42)
# y_transformed = qt.fit_transform(y.values.reshape(-1, 1)).ravel()
# 
# # Разделение на обучающую и тестовую выборки
# X_train, X_test, y_qt_train, y_qt_test = train_test_split(
#     X, y_transformed, test_size=0.3, random_state=RANDOM_STATE
# )
# 
# # Обучение модели
# final_pipeline.fit(X_train, y_qt_train)
# 
# # Предсказание 
# y_qt_pred = final_pipeline.predict(X_test)
# 
# # Обратное преобразование для возврата к исходной шкале
# y_pred = qt.inverse_transform(y_qt_pred.reshape(-1, 1)).ravel()
# y_test_original = qt.inverse_transform(y_qt_test.reshape(-1, 1)).ravel()
# 
# # Расчет метрик на исходной шкале
# print("RMSE:", root_mean_squared_error(y_test_original, y_pred))
# print("R²:", r2_score(y_test_original, y_pred))

# Еще хуже, чем у log1p показатели
# RMSE: 25475.711594983382
# R²: 0.9069929548806451

In [23]:
# from scipy import stats
# import numpy as np
# 
# # Выявление и обработка выбросов в признаках
# def remove_outliers_safe(df, y, threshold=3):
#     """
#     Удаляет выбросы на основе z-score, но более безопасным способом
#     """
#     df_clean = df.copy()
#     y_clean = y.copy()
#     
#     # Вычисляем z-score для каждого столбца
#     outliers_mask = np.zeros(len(df_clean), dtype=bool)
#     
#     for col in df_clean.select_dtypes(include=['float64', 'int64']).columns:
#         # Проверяем, что в столбце достаточно уникальных значений для z-score
#         if len(df_clean[col].unique()) > 5:  # Защита от категориальных данных с числовыми кодами
#             # Используем try-except для защиты от ошибок при расчете z-score
#             try:
#                 # Игнорируем NaN значения при расчете z-score
#                 with np.errstate(all='ignore'):  # Подавляем предупреждения
#                     z_scores = np.abs(stats.zscore(df_clean[col], nan_policy='omit'))
#                     col_outliers = np.logical_and(~np.isnan(z_scores), z_scores > threshold)
#                     outliers_mask = np.logical_or(outliers_mask, col_outliers)
#             except:
#                 print(f"Не удалось рассчитать z-score для столбца {col}. Пропускаем.")
#                 continue
#     
#     # Процент выбросов
#     outlier_percentage = outliers_mask.mean() * 100
#     print(f"Обнаружено {outlier_percentage:.2f}% выбросов.")
#     
#     # Если процент выбросов слишком велик, ограничиваемся верхним пределом
#     max_outlier_percentage = 20  # Максимальный % строк, которые можно удалить
#     if outlier_percentage > max_outlier_percentage:
#         print(f"Слишком много выбросов! Ограничиваем удаление до {max_outlier_percentage}%")
#         # Выбираем только самые экстремальные выбросы
#         z_scores_all = np.zeros((len(df_clean), len(df_clean.select_dtypes(include=['float64', 'int64']).columns)))
#         
#         for i, col in enumerate(df_clean.select_dtypes(include=['float64', 'int64']).columns):
#             if len(df_clean[col].unique()) > 5:
#                 try:
#                     z_scores_all[:, i] = np.abs(stats.zscore(df_clean[col], nan_policy='omit'))
#                 except:
#                     continue
#         
#         # Максимальное отклонение по z-score для каждой строки
#         max_z_scores = np.nanmax(z_scores_all, axis=1)
#         # Сортируем индексы по убыванию z-score
#         sorted_indices = np.argsort(-max_z_scores)
#         # Выбираем индексы верхних n% строк для удаления
#         n_to_remove = int(len(df_clean) * max_outlier_percentage / 100)
#         indices_to_remove = sorted_indices[:n_to_remove]
#         outliers_mask = np.zeros(len(df_clean), dtype=bool)
#         outliers_mask[indices_to_remove] = True
#     
#     # Удаляем выбросы
#     df_cleaned = df_clean[~outliers_mask].reset_index(drop=True)
#     y_cleaned = y_clean[~outliers_mask].reset_index(drop=True)
#     
#     print(f"Размер исходной выборки: {len(df_clean)}")
#     print(f"Размер очищенной выборки: {len(df_cleaned)}")
#     
#     return df_cleaned, y_cleaned
# 
# # Применяем функцию
# X_clean, y_clean = remove_outliers_safe(X, y, threshold=3)
# X, y = X_clean, y_clean

# Выводы: очистка аномальных данных урезала дата-сет, упростила его, поэтому показатели улучшились сильно, даже на CV, но не на сабмишне, где показатели ухудшились. Типичный оверфиттинг.

In [24]:
# Катбуст сам выбирает даныне для валидации, поэтому отсекаем данные только для теста
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# Проверка размера полученных наборов
print(f'Train size: {len(X_train)}')
print(f'Test size: {len(X_test)}')

Train size: 1022
Test size: 438


In [25]:
mlflow.set_experiment("Advanced Feature Engineering")
with mlflow.start_run():
    final_pipeline.fit(X_train, y_train)
    results = mm.evaluate_regression(final_pipeline, X_test, y_test)
    
    mlflow.log_metric("RMSE_manual", results['RMSE'])
    mlflow.log_metric("MAE_manual", results['MAE'])
    mlflow.log_metric("MAPE_manual", results['MAPE'])

2025/04/02 11:19:16 INFO mlflow.tracking.fluent: Experiment with name 'Advanced Feature Engineering' does not exist. Creating a new experiment.


R2 Score: 0.9261
MSE:      515628859.6444
RMSE:     22707.4626
MAE:      13695.7936
MSLE:     0.0141
RMSLE:    0.1187
MAPE:     0.0817


Стремимся к:
- R2 Score: 0.9261
- MSE:      515628859.6444
- RMSE:     22707.4626
- MAE:      13695.7936
- MSLE:     0.0141
- RMSLE:    0.1187
- MAPE:     0.0817

In [18]:
# Создаем объект многократной кросс-валидации
# n_splits=5 - количество фолдов
# n_repeats=3 - количество повторений всей процедуры
# random_state для воспроизводимости результатов
repeated_cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=RANDOM_STATE)

# Запускаем кросс-валидацию
scores = cross_val_score(final_pipeline, X, y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=repeated_cv)

# Преобразуем отрицательные значения в положительные
rmse_scores = -scores

# Выводим результаты
print(f"Все значения RMSE: {rmse_scores}")
print(f"Средний RMSE на многократной кросс-валидации: {rmse_scores.mean():.4f}")
print(f"Стандартное отклонение RMSE: {rmse_scores.std():.4f}")

KeyboardInterrupt: 

Стремимся к
- Все значения RMSE: [25419.11631511 23590.88424668 40286.11946863 26251.67462774
 17518.97965098 25570.05300749 25002.81419282 20915.43518056
 24727.88011941 31999.84374666 31155.31957707 21067.03441678
 29403.32588714 18900.17080983 25119.71807368]
- Средний RMSE на многократной кросс-валидации: 25795.2246
- Стандартное отклонение RMSE: 5537.2770

## Получили исходный результат выше, который хотим улучшить. Начинаем эксперименты.

Пытаемся объединить фичи, которые явно не особо имеют смысла идти по отдельности, например: 
- BsmtFinSF1; BsmtFinSF2; BsmtUnfSF – кажется можно удалить и оставить только TotalBsmtSF
- 1stFlrSF; 2ndFlrSF - кажется, можно заменить на сумму этих значений.

Удаление площадей подвалов ничего не дало, кроме ухудшений.
Добавление суммы с последующим удаление колонок - тоже ничего не дало.


Экспериментировал с регуляризацией catboost для уменьшения оверфиттинга - результат не изменился. Ниже испробованные конфигурации:
params_1 = {
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'RMSE',
    'l2_leaf_reg': 3,
    'rsm': 0.8,
    'random_seed': RANDOM_STATE,
    'verbose': 0
}
params_2 = {
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'RMSE',
    'l2_leaf_reg': 10,
    'rsm': 0.8,
    'random_seed': RANDOM_STATE,
    'verbose': 0
}
params_4 = {
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 7,
    'loss_function': 'RMSE',
    'l2_leaf_reg': 5,
    'rsm': 0.8,
    'bagging_temperature': 1.0,
    'random_strength': 1.5,
    'random_seed': RANDOM_STATE,
    'verbose': 0
}
params_5 = {
    'iterations': 1000,
    'learning_rate': 0.03,
    'depth': 6,
    'loss_function': 'RMSE',
    'l2_leaf_reg': 10,
    'rsm': 0.8,
    'bagging_temperature': 2.0,
    'random_strength': 2.0,
    'random_seed': RANDOM_STATE,
    'verbose': 0
}

Выше пробовали сгенерировать синтетическе данные - результат значительно (!) ухудшился на алгоритме synth_generator.
На synth_generator_gaussian - результаты на малых синт. данных были схожими, но на больших - значительно хуже.
CTGAN - не запустился.

Добавление новых фич, без удаления старых - тоже не дало почти никакого улучшения. Только легкое ухудшение.

Нормализация y (log1p, sqrt, boxcox) - не дало улучшения.
Удаление выбросов из всех данных - дало сильное улучшение, даже на cv, но на сабмишне - ухудшение. Оверфиттинг потому что получился.

Получил небольшой прирост по RMSE, когда заменил onehot на target encoding и подобрал параметры энкодинга. По итогу вернулся к onehot  - он стабильнее.

Еще улучшение небольшое при добавлении двух новых фичей в паре
- X['Neighborhood_Zoning'] = X['Neighborhood'] + '_' + X['MSZoning']
- X['SaleType_Condition'] = X['SaleType'] + '_' + X['SaleCondition']

СУЩЕСТВЕННОЕ улучшение (400 по RMSE) при добавлении единого числового индекса по качественным призанкам
>quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}

>X['TotalQualScore'] = (
>    X['ExterQual'].map(quality_dict) +
>    X['KitchenQual'].map(quality_dict) +
>    X['BsmtQual'].map(quality_dict) +
>    X['HeatingQC'].map(quality_dict) +
>    X['GarageQual'].map(quality_dict) +
>    X['FireplaceQu'].map(quality_dict)
>)


### ИТОГО: 
очень сильный прирост (почти +1к к RMSE) получили благодаря комбинации (!) новых признаков. И submission улучшился до 12877, что почти на 400 пунктов лучше прошлого сабмишна самого лучшего. Оставляем эти изменения. Все остальное - комментим.

Небольшое улучшение от параметров вот таких: default_params = {
    'iterations': 2000, 
    'learning_rate': 0.03, 
    'depth': 5,
    'l2_leaf_reg': 3,  # Можно варьировать (1, 3, 5, 7, 9)
    'bagging_temperature': 1,
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 100,
} добавили: 'l2_leaf_reg': 3, ; 'learning_rate': 0.03; 'iterations': 2000, НО на сабмишне хуже - поэтому откат.

Добавление полиномиальных фич очень хорошо улучшило результаты (на тысячу + почти), но показатели на каггл плохие при самбишне - 13600, а было без полиномиальных 12877. На лицо, в очередной раз, оверфиттинг. Может, стоит поиграться параметром l2 регуляризации catboost?

Винзоризация дает небольшое улучшение в +600 по RMSE, но при самбишне - ухудшение. Поэтому убираем.
Winsorizing "срезает" хвосты распределения. Хотя это может помочь модели лучше подогнаться под основную массу данных в тренировочном наборе (уменьшая RMSE), эти экстремальные значения в тестовом наборе могут нести важную информацию. Возможно, очень большие дома (GrLivArea) или участки (LotArea) действительно имеют непропорционально высокую цену, и, ограничивая их значения сверху, мы заставляем модель недооценивать такие объекты в тесте.

Пробовал возвращать удаленные интуитивно поля и экспериментировать с ними - ничего, кроме ухудшения не дало.

## 4. Формируем Submission файл для Kaggle обучив модель на 100% данных

In [45]:
# final_pipeline.fit(X, y_log)
# final_pipeline.fit(X, y_sqrt)
final_pipeline.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   IterativeImputer(estimator=RandomForestRegressor(n_estimators=50,
                                                                                                                    random_state=42),
                                                                                    random_state=42))]),
                                                  Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplac...
       'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC',
       'HouseStyle', 'KitchenQual', 'MSZoning', 'MasVnrType', 'Neighborhood',
       'Neighborhood_Zoning', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'SaleType_Condition', 'Street',
       'Utilities'],
      dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x30e9ee870>)])

In [46]:
X_submission = test_data.drop(columns=bad_columns)
X_submission.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition1,Condition2,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,AllPub,NAmes,Feedr,Norm,...,0,0,120,0,NaN,MnPrv,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,AllPub,NAmes,Norm,Norm,...,0,0,0,0,NaN,NaN,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,AllPub,Gilbert,Norm,Norm,...,0,0,0,0,NaN,MnPrv,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,AllPub,Gilbert,Norm,Norm,...,0,0,0,0,NaN,NaN,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,AllPub,StoneBr,Norm,Norm,...,0,0,144,0,NaN,NaN,1,2010,WD,Normal


In [47]:
# # Делаем теже преобразования с X_submission, что и с X
# X_submission['TotalSF'] = X_submission['1stFlrSF'] + X_submission['2ndFlrSF'] + X_submission['TotalBsmtSF']
# X_submission['Age'] = X_submission['YrSold'] - X_submission['YearBuilt']
# X_submission['QualityIndex'] = X_submission['OverallQual'] * X_submission['OverallCond']
# X_submission['HasPool'] = (X_submission['PoolArea'] > 0).astype(int)
# X_submission['Remodeled'] = (X_submission['YearRemodAdd'] != X_submission['YearBuilt']).astype(int)
# X_submission['TotalBathrooms'] = X_submission['FullBath'] + 0.5 * X_submission['HalfBath'] + X_submission['BsmtFullBath'] + 0.5 * X_submission['BsmtHalfBath']
# X_submission.shape

X_submission['Neighborhood_Zoning'] = X_submission['Neighborhood'] + '_' + X_submission['MSZoning']
X_submission['SaleType_Condition'] = X_submission['SaleType'] + '_' + X_submission['SaleCondition']
X_submission['TotalQualScore'] = (
    X_submission['ExterQual'].map(quality_dict) +
    X_submission['KitchenQual'].map(quality_dict) +
    X_submission['BsmtQual'].map(quality_dict) +
    X_submission['HeatingQC'].map(quality_dict) +
    X_submission['GarageQual'].map(quality_dict) +
    X_submission['FireplaceQu'].map(quality_dict)
)
X_submission['PorchDeckArea'] = (X_submission['WoodDeckSF'] + X_submission['OpenPorchSF'] + X_submission['EnclosedPorch'] +
                      X_submission['3SsnPorch'] + X_submission['ScreenPorch'])
X_submission['HasFireplace'] = (X_submission['Fireplaces'] > 0).astype(int)
X_submission['HasGarage'] = (~X_submission['GarageType'].isna()).astype(int)
X_submission['HasFence'] = (~X_submission['Fence'].isna()).astype(int)
X_submission['HasPorchDeck'] = (X_submission['PorchDeckArea'] > 0).astype(int)
X_submission.shape

(1459, 81)

In [48]:
# ОЧЕНЬ ВАЖНО убедиться, что набор признаков (X_submission) абсолютно совпадает с X по количеству и порядку колонок!
# Приводим колонки точно к нужному порядку:
X_submission = X_submission.reindex(columns=feature_cols)

In [49]:
y_submission = final_pipeline.predict(X_submission)
y_submission

array([121982.72314189, 157490.60448354, 189784.29652449, ...,
       165699.65513501, 117650.3575256 , 223710.15453793])

In [50]:
# Обратное преобразование для возврата к исходной шкале
# y_submission = np.expm1(y_submission)
# y_submission = y_submission ** 2
# y_submission

In [51]:
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': y_submission
})
submission.set_index('Id', inplace=True)
submission.head()

,SalePrice
Id,
1461,121982.723142
1462,157490.604484
1463,189784.296524
1464,192438.774384
1465,176565.159645


In [52]:
submission.to_csv('submission_final_with_new_features_v4.7.csv')